# WFI Reference Pixel Reference File Pipeline Example

This notebook shows how to use the WFI Reference File Pipeline to create reference pixel reference files to submit to CRDS. At this stage, it is in the testing phase, and this notebook is used to assemble the steps of the pipeline to make test data.

Presently, the input files come from TVAC1 .asdf files. These data are located on central storage at:

```
/grp/roman/GROUND_TESTS/TVAC1/ASDF/NOM_OPS/OTP00639_All_TV1a_R1_MCEB/Activity_1
```

# Some imports ... 

In [1]:

import glob
import os

from wfi_reference_pipeline.reference_types.referencepixel.referencepixel import (
    ReferencePixel,
)
from wfi_reference_pipeline.resources.make_dev_meta import MakeDevMeta
from wfi_reference_pipeline.utilities import logging_functions

logging_functions.configure_logging('wfi_refpixel_example')

# Reference Pixel Files

To make a reference pixel reference file, we need to perform the Improved Roman Reference Correction (Rauscher et al., in prep), We'll use several darks to compute a model of the read noise in the normal pixels that is a linear combination of the reference output, left, and right column pixels (IRRC; Rauscher et al., in prep).  The sums for each exposure are then combined/summed together to create a final model that minimizes the 1/f noise given as
        $Fn = \alpha Fa + \gamma Fl + \zeta Fr$.  
 The coefficients are then saved as a final reference class attribute.  

In [2]:
# Set the directory of the TVAC1 data.
tvac1_dir = '/grp/roman/GROUND_TESTS/TVAC1/ASDF/NOM_OPS/OTP00639_All_TV1a_R1_MCEB/Activity_1'
print(f'Using directory {tvac1_dir} for TVAC1 data.')

Using directory /grp/roman/GROUND_TESTS/TVAC1/ASDF/NOM_OPS/OTP00639_All_TV1a_R1_MCEB/Activity_1 for TVAC1 data.


In [3]:
# Grab the files.
dark_files = glob.glob(os.path.join(tvac1_dir, '*WFI09*.asdf'))[0:2]

# print file names
print('Files used for reference pixel creation:')
for df in dark_files:
    print(f'\t{os.path.basename(df)}')
  

Files used for reference pixel creation:
	TVAC1_NOMOPS_TTNOISE_20231021062643_WFI09_uncal.asdf
	TVAC1_NOMOPS_TTNOISE_20231021075323_WFI09_uncal.asdf


We can also create the reference files using already extracted data in either a Quantity or np.ndarray format.  In this case, we would provide either a 3D (one exposure) or 4D (multiple exposures) array. 

## Setup 

The example refpix reference file will be created in the RFP scratch directory. This directory has it contents automatically deleted every night. A default outfile name is given and if clobber set to True will be overwritten.

**tmppath**: The intermediatary files will be temporarily saved during the IRRC.  
- state the path for temporary sums to save in. The path must fully exist.  It will create a folder inside called 'tmp_IRRC_[detector]'
- can use tmppath = None, and the tmp_IRRC_[detector] will save in the current working directory

**outfile**: filename for reference pixel coefficient file

In [4]:
sca = 'WFI09'  # state which detector is being used.

write_path = '/grp/roman/RFP/DEV/scratch/'  # Set the write path to be in the RFP scratch directory.
tmppath = write_path # use the same folder to save the temporary sums folder
outfile = write_path + f'roman_referencepixel_dev_{sca}.asdf' # Default reference pixel development filename.

# Meta Data

The RFP utilizes python data classes to construct reference file type dependent meta data for development from the "ref_type" keyword, which is then exported into a dictionary for valid usage with roman data models. 

In [5]:
print("Generating ReferencePixel Metadata")
tmp = MakeDevMeta(ref_type='REFPIX')  # Get the default reference file type specific development meta data.
refpixel_dev_meta = tmp.meta_referencepixel  # Export the meta data as a dictionary.

Generating ReferencePixel Metadata


In [6]:
print(refpixel_dev_meta)  # Inspect default development meta data.

WFIMetaReferencePixel(reference_type='REFPIX', pedigree='DUMMY', description='For RFP Development and DMS Build Support.', author='RFP', _use_after=<Time object: scale='utc' format='isot' value=2023-01-01T00:00:00.000>, telescope='ROMAN', origin='STSCI', instrument='WFI', instrument_detector='WFI01', input_units=Unit("DN"), output_units=Unit("DN"))


# Make the referencepixel coefficients via the ReferencePixel class

We now have the files and folder paths of darks for one detector. We can pass this list of images to the ReferencePixel class to make ASDF reference pixel reference files for Roman.

Note: this step is relatively slow.  Be patient.

In [7]:
 
refpix_ref = ReferencePixel(meta_data=refpixel_dev_meta,
                                           file_list=dark_files,
                                           outfile=outfile,
                                           clobber=True)
refpix_ref.make_referencepixel_image(tmppath=tmppath)


If we provided a data array, we would instead set up the ReferencePixel class as below.  Instead of ```file_list```, we provide a ```ref_type_data```.  
Note: we must provide a detector_name for this data since there is no associated metadata provided from an input file.  This is key for both the reference file meta data and the filename. 

```
refpix_ref = ReferencePixel(meta_data=refpixel_dev_meta,
                                           ref_type_data=input_data_array,
                                           outfile=outfile,
                                           clobber=True)
refpix_ref.make_referencepixel_image(tmppath=tmppath, detector_name = sca)
```

## Build the datamodel for the reference file, write it to disk, and change the permissions of the file. 

In [9]:
# save to .asdf file
refpix_ref.generate_outfile()
os.chmod(outfile, 0o666)  # Set file permissions to read+write for owner, group, and global.
print('Made file -> ', outfile)

Made file ->  /grp/roman/RFP/DEV/scratch/roman_referencepixel_dev_WFI09.asdf
